In [1]:
import libsbml as sb # SBML integration
import numpy as np # for making arrays
import pandas as pd # for dataframe conversion

In [2]:
# pulling data from the SBML file
reader = sb.SBMLReader() # open the reader
document = reader.readSBMLFromFile("iJO1366_e_coli.xml") # read the file into document
model = document.getModel() # and extract the model from the file

In [3]:
## Making Connectivity Matrix

# setting up
specieslist = model.getListOfSpecies() # get a list of all the species
rxnslist = model.getListOfReactions() # and of all the reactions
numspecies = len(specieslist.getListOfAllElements()) # number of species (non-unique; if 1 species is in two compartments it is recorded twice with 2 different SIDs)
numrxns = 2582 # from literature (similarly non-unique)
speciesIDs = np.empty(numspecies,dtype=object) # prepare an array that stores only the IDs of every species
for s in range(numspecies):
    speciesIDs[s] = specieslist.get(s).getId() # and add in all the IDs


connect = np.zeros((numspecies,numspecies)) # make an N x N connectivity matrix where N is the number of species/substrates. we will have that if two substrates with speciesIDs indices i and j are linked, connect[i,j] = connect[j,i] = 1; otherwise 0.
for r in range(numrxns): # for every reaction
    rxn = rxnslist.get(r)
    numreacts = rxn.getNumReactants() # find the number of reactants
    numprods = rxn.getNumProducts() # and products
    if numreacts+numprods>1: # if there are links to record...
        substrates = np.empty(numreacts+numprods,dtype=int) # the total number of involved substrates will be reactants+products
        if numreacts>0: # if reactants are involved...
            for i in range(numreacts): # for every reactant...
                species = rxn.getReactant(i).getSpecies() # store its species id
                speciesindex = np.where(speciesIDs == species)[0][0] # store the corresponding speciesIDs index
                substrates[i] = speciesindex # add the speciesIDs index to the list of substrates
        if numprods>0: # now do the same for products
            for i in range(numprods):
                species = rxn.getProduct(i).getSpecies()
                speciesindex = np.where(speciesIDs == species)[0][0]
                substrates[i+numreacts] = speciesindex
        for i in np.arange(len(substrates)): # for each recorded substrate...
            linkindexes = np.delete(substrates,i) # make a list of the other recorded substrates
            for n in np.arange(len(linkindexes)): # and for each of them...
                connect[substrates[i],linkindexes[n]] = 1 # put a '1' in the linkage cell corresponding to the two linked substrates

In [4]:
connectpd = pd.DataFrame(connect) # convert to dataframe for file saving
# connectpd.to_csv("C:/Users/camer/OneDrive - The University of Chicago/College/Classes/BIOS 26211/final project/iJO1366 E coli model/connectivity.csv") # save the file!

In [6]:
## Performing Topological Reduction

specieslinks = np.zeros(len(speciesIDs))
for row in np.arange(len(connect[:,0])):
    specieslinks[row] = sum(connect[row,:])

nodecolors = np.empty(len(specieslinks),dtype=str)

nodecolors[:] = 'a' # set all nodes to black

for end in np.where(specieslinks==1)[0]: # for each node with only 1 link...
    prevnode = end
    node = np.where(connect[end,:]==1)[0][0] # find and jump to the single linked node
    nodelinkcount = specieslinks[node] # count the number of links
    while nodelinkcount==2: # as long as the present node has 2 links
        nodecolors[node] = 'g' # color the node green
        nodelinks = np.where(connect[node,:]==1)[0] # find the two connections
        nextnode = nodelinks[nodelinks != prevnode] # target the node you haven't been to yet
        prevnode = node # make the jump !
        node = nextnode
        nodelinkcount = specieslinks[node] # see how many links the new node has. if it's 2, repeat; if not, find the next node with only 1 link and start over

for node in np.where(specieslinks==2)[0]: # look at all the 2-link nodes
    if nodecolors[node] != 'g': # if it's not green
        nodecolors[node] = 'u' # make it blue

# coloring branching hairs green
for bluenode in np.where(nodecolors=='u')[0]: # for each blue node
    if nodecolors[bluenode] != 'u': continue # make sure the node is still blue; if not, move on
    nodelinks = np.where(connect[bluenode,:]==1)[0] # find its links
    blueblacklinks = np.empty(0)
    for link in nodelinks:
        if nodecolors[link]=='u' or nodecolors[link]=='a': blueblacklinks = np.append(blueblacklinks,link)
    
    if len(blueblacklinks>0): # if any of the links are blue or black...
        blueblacklink = blueblacklinks[0] # pick the first one as the link
        connect[bluenode,blueblacklink] = connect[blueblacklink,bluenode] = 0 # sever the link

        # burning algorithm
        burnlabels = np.repeat(-1,len(speciesIDs)) # label all nodes -1
        burnlabels[bluenode] = 0 # except the starting node
        nodes = np.where(connect[bluenode,:]==1)[0] # find its singular remaining link
        stage = 1 # set the stage
        while len(nodes) > 0: # as long as you still have more nodes linked
            burnlabels[nodes] = stage # assign the nodes labels based on the current stage
            nextnodes = np.empty(0) # initialize a list for the next stage's nodes
            for node in nodes: # for each node in the stage...
                if nodelinks[node]>0: # if it has links...
                    nextnodes = np.append(nextnodes,np.where(connect[node,:]==1)[0]) # append its linked nodes to the list
            nodes = nextnodes # move to the next stage
            stage = stage+1 # and update the stage count

        if burnlabels[blueblacklink] > 0: # if the node on the other side of the severed link was burned (i.e. is still in the starting node's network)
            continue # it's not a hair separation! make no changes and move on
        else: # if we DID separate a hair
            burnedsize = len(burnlabels[burnlabels != -1])
            if burnedsize < 0.5*(len(burnlabels[burnlabels == -1])): # if the burned part is less than half the size of the unburned part (i.e., it's a hair)...
                nodecolors[burnlabels != -1] = 'g' # make the burned nodes green
            else: # if the burned part is more than half the size of the unburned part (i.e., it's the network)
                # repeat the burning algorithm, but on the other side of the severed link
                burnlabels = np.repeat(-1,len(speciesIDs))
                burnlabels[blueblacklink] = 0
                nodes = np.where(connect[blueblacklink,:]==1)[0]
                stage = 1
                while len(nodes) > 0:
                    burnlabels[nodes] = stage
                    nextnodes = np.empty(0)
                    for node in nodes:
                        if nodelinks[node]>0:
                            nextnodes = np.append(nextnodes,np.where(connect[node,:]==1)[0])
                    nodes = nextnodes
                    stage = stage+1
                
                nodecolors[burnlabels != -1] = 'g' # and make the new burned nodes green

        connect[bluenode,blueblacklink] = connect[blueblacklink,bluenode] = 1 # finally, retie the severed link
    # repeat for all blue nodes
    
nodecolors[nodecolors == 'a'] = 'r' # make all the still-black nodes red

# Note: for the two reduction steps below, I'm not sure what the supplemental material (p. 13) means by "store [the removed hairs/arcs] as separate small networks together with the label of the [node/two ends] they connect to." I'll skip this for now and see what it affects.

# Half-reduction (removing hairs)
connect_halfreduced = connect # make a copy of the adjacency matrix for the half-reduction
connect_halfreduced[nodecolors=='g', :] = 0 # sever all links with green nodes
connect_halfreduced[:,nodecolors=='g'] = 0 # ^
nodecolors_halfreduced = nodecolors # make an updated coloring...
nodecolors_halfreduced[nodecolors=='g'] = '' # ...where green nodes are colorless

# Full Reduction (removing arcs)
connect_reduced = connect_halfreduced # make a copy of the half-reduced adjacency matrix for the full-reduction
nodecolors_reduced = nodecolors_halfreduced # same for coloring
for bluenode in np.where(nodecolors_reduced=='u')[0]: # for each blue node
    if nodecolors_reduced[bluenode] != 'u': continue # make sure the node is still blue; if not, move on
    node = bluenode
    nodelinks = np.where(connect[bluenode,:]==1)[0]
    prevnode = nodelinks[0]
    redlinks = np.repeat(-1,2) # initialize the vector holding the two red ends of the arc
    while redlinks[1]<0: # as long as there is still an end to be found...
        nodelinks = np.where(connect[node,:]==1)[0] # find the current node's links
        nodelinkcolors = nodecolors[nodelinks] # and their colors
        if any(nodelinkcolors=='r'): # check to see if the node is linked to any red nodes. if so...
            rednodelinks = nodelinks[np.where(nodelinkcolors=='r')[0]] # store any linked nodes that are red
            if redlinks[0]<0: # if we haven't found the first arc-end yet
                redlinks[0] = rednodelinks[0] # store a red node as the first arc-end
            else: redlinks[1] = rednodelinks[0] # if we have found the first arc-end, store the red node as the second end
            if len(rednodelinks)==2: # if the blue node is linked to 2 red nodes (i.e., it's a 1-node arc)...
                redlinks[1] = rednodelinks[1] # store the other red node as the second arc-end
                nodecolors_reduced[node] = 'p' # make the blue node purple
                break # and break out of the while loop
            else: 
                prevnode = redlinks[0] # otherwise, prep to move in the other direction
        else:
            nextnode = nodelinks[nodelinks != prevnode]
            nodecolors_reduced[node] = 'p'
            prevnode = node
            node = nextnode
    # now that both red arc-ends have been found...
    connect_reduced[redlinks[0],redlinks[1]] = connect_reduced[redlinks[1],redlinks[0]] = 1 # link the two ends
    # and move onto the next blue node
nodecolors_reduced[nodecolors_reduced=='p'] = '' # make all purple (formerly blue) nodes colorless

# question: where do the separated arcs and hairs factor back in? do they not factor into the clustering of figs 4A-C, only being involved in the 4D graphic?
# for now i'll just assume they're discarded except for fig 4D:

# remove all rows/columns of adjacency matrix pertaining to removed nodes
connect_halfreduced = connect_halfreduced[np.where(nodecolors_halfreduced != '')[0],np.where(nodecolors_halfreduced != '')[0]]
connect_reduced = connect_reduced[np.where(nodecolors_reduced != '')[0],np.where(nodecolors_reduced != '')[0]]

# replace all np.where(...)[0] to np.int64(np.where(...)[0]) so they can be used as indices

[259. 466.] 259.0


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [11]:
# test = np.array([2, 5, 7, 9])
# testcol = np.array(['a','a','d','a', 'a', 'a', 'b', 'b', 'a', 'b'])
# subtest = np.empty(0)
# print(subtest)
# for i in test:
#     if testcol[i]=='b': subtest = np.append(subtest,i)
# print(subtest)
# test[test != 5]



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices